In [ ]:
# Object-Oriented Programming in Python

A comprehensive guide to classes, objects, and OOP principles in Python.

## Table of Contents
1. [Class Design and Implementation](#class-design)
2. [Inheritance and Polymorphism](#inheritance)
3. [Encapsulation and Data Hiding](#encapsulation)
4. [Abstract Classes and Interfaces](#abstract-classes)
5. [Design Patterns](#design-patterns)

Each section includes:
- 📊 Performance Analysis
- 🎯 Visual Class Diagrams
- 💻 Implementation Examples
- ⚡ Best Practices
- 📝 Practice Problems
- 🎯 Interview Questions


In [ ]:
class Vehicle:
    """A base class representing a vehicle with basic attributes and methods."""
    
    def __init__(self, brand: str, model: str, year: int):
        self.brand = brand
        self.model = model
        self.year = year
        self._mileage = 0
    
    @property
    def mileage(self) -> float:
        """Get the current mileage of the vehicle."""
        return self._mileage
    
    @mileage.setter
    def mileage(self, value: float) -> None:
        """Set the mileage of the vehicle."""
        if value < 0:
            raise ValueError("Mileage cannot be negative")
        self._mileage = value
    
    def get_info(self) -> str:
        """Return a string containing vehicle information."""
        return f"{self.year} {self.brand} {self.model} - {self.mileage} miles"
    
    def __str__(self) -> str:
        return self.get_info()


In [ ]:
# 1. Class Design and Implementation

## Example: Bank Account System

### Class Diagram
```
+-------------------+
|    BankAccount    |
+-------------------+
| - account_number  |
| - balance        |
| - owner_name     |
| - account_type   |
+-------------------+
| + deposit()      |
| + withdraw()     |
| + get_balance()  |
| + transfer()     |
+-------------------+

Performance Analysis:
- Method calls: O(1)
- Memory usage: O(1) per instance
- Transaction logging: O(n) where n is transaction count
```

### Implementation Features:
- Type hints and data validation
- Property decorators for encapsulation
- Transaction history tracking
- Exception handling for edge cases


In [ ]:
from typing import List, Optional, Dict
from datetime import datetime
from dataclasses import dataclass
from enum import Enum
import uuid

class AccountType(Enum):
    SAVINGS = "savings"
    CHECKING = "checking"
    BUSINESS = "business"

@dataclass
class Transaction:
    timestamp: datetime
    transaction_type: str
    amount: float
    balance_after: float

class BankAccount:
    def __init__(self, owner_name: str, account_type: AccountType, initial_balance: float = 0):
        self._account_number = str(uuid.uuid4())
        self._balance = initial_balance
        self._owner_name = owner_name
        self._account_type = account_type
        self._transactions: List[Transaction] = []
        self._log_transaction("OPEN", initial_balance)
    
    @property
    def balance(self) -> float:
        return self._balance
    
    @property
    def account_number(self) -> str:
        return self._account_number
    
    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self._balance += amount
        self._log_transaction("DEPOSIT", amount)
    
    def withdraw(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        self._balance -= amount
        self._log_transaction("WITHDRAW", amount)
    
    def transfer(self, target_account: 'BankAccount', amount: float) -> None:
        if amount <= 0:
            raise ValueError("Transfer amount must be positive")
        if amount > self._balance:
            raise ValueError("Insufficient funds for transfer")
        
        self.withdraw(amount)
        target_account.deposit(amount)
        self._log_transaction("TRANSFER_OUT", amount)
    
    def get_transaction_history(self) -> List[Transaction]:
        return self._transactions.copy()
    
    def _log_transaction(self, transaction_type: str, amount: float) -> None:
        transaction = Transaction(
            timestamp=datetime.now(),
            transaction_type=transaction_type,
            amount=amount,
            balance_after=self._balance
        )
        self._transactions.append(transaction)
    
    def __str__(self) -> str:
        return f"{self._account_type.value.title()} Account - {self._owner_name} (Balance: ${self._balance:.2f})"

# Example usage
savings = BankAccount("John Doe", AccountType.SAVINGS, 1000)
checking = BankAccount("John Doe", AccountType.CHECKING, 500)

# Perform some transactions
savings.deposit(500)
savings.withdraw(200)
savings.transfer(checking, 300)

# Print account information and transaction history
print(savings)
print(checking)
print("\nTransaction History:")
for transaction in savings.get_transaction_history():
    print(f"{transaction.transaction_type}: ${transaction.amount:.2f} - Balance: ${transaction.balance_after:.2f}")


In [ ]:
# 2. Inheritance and Polymorphism

## Example: Shape Hierarchy System

### Class Diagram
```
       +-------------+
       |   Shape     |
       +-------------+
       | - color     |
       | + area()    |
       | + perimeter()|
       +-------------+
             ↑
    +--------+--------+
    |                 |
+--------+      +---------+
| Circle |      | Polygon |
+--------+      +---------+
| - radius|      | - sides |
+--------+      +---------+
                     ↑
            +--------+--------+
            |                 |
      +---------+      +----------+
      | Triangle |      | Rectangle|
      +---------+      +----------+

Performance Implications:
- Method resolution: O(d) where d is inheritance depth
- Memory overhead: Additional attributes per level
- Virtual method calls: Slight performance impact
```

### Implementation Features:
- Abstract base classes
- Method overriding
- Interface segregation
- Duck typing principles


In [ ]:
from abc import ABC, abstractmethod
import math
from typing import List, Optional
from dataclasses import dataclass

class Shape(ABC):
    def __init__(self, color: str):
        self.color = color
    
    @abstractmethod
    def area(self) -> float:
        """Calculate the area of the shape."""
        pass
    
    @abstractmethod
    def perimeter(self) -> float:
        """Calculate the perimeter of the shape."""
        pass
    
    def __str__(self) -> str:
        return f"{self.__class__.__name__} (Color: {self.color})"

class Circle(Shape):
    def __init__(self, radius: float, color: str):
        super().__init__(color)
        if radius <= 0:
            raise ValueError("Radius must be positive")
        self.radius = radius
    
    def area(self) -> float:
        return math.pi * self.radius ** 2
    
    def perimeter(self) -> float:
        return 2 * math.pi * self.radius

class Polygon(Shape):
    def __init__(self, sides: List[float], color: str):
        super().__init__(color)
        if any(side <= 0 for side in sides):
            raise ValueError("All sides must be positive")
        self.sides = sides
    
    def perimeter(self) -> float:
        return sum(self.sides)

class Triangle(Polygon):
    def __init__(self, side1: float, side2: float, side3: float, color: str):
        super().__init__([side1, side2, side3], color)
        if not self._is_valid_triangle():
            raise ValueError("Invalid triangle sides")
    
    def _is_valid_triangle(self) -> bool:
        a, b, c = self.sides
        return (a + b > c) and (b + c > a) and (a + c > b)
    
    def area(self) -> float:
        # Using Heron's formula
        s = self.perimeter() / 2
        a, b, c = self.sides
        return math.sqrt(s * (s - a) * (s - b) * (s - c))

class Rectangle(Polygon):
    def __init__(self, width: float, height: float, color: str):
        super().__init__([width, height, width, height], color)
        self.width = width
        self.height = height
    
    def area(self) -> float:
        return self.width * self.height

# Example usage and polymorphism
shapes: List[Shape] = [
    Circle(5, "red"),
    Triangle(3, 4, 5, "blue"),
    Rectangle(4, 6, "green")
]

# Demonstrate polymorphic behavior
for shape in shapes:
    print(f"\n{shape}")
    print(f"Area: {shape.area():.2f}")
    print(f"Perimeter: {shape.perimeter():.2f}")

# Performance demonstration
import time

def measure_method_resolution(shape: Shape, iterations: int = 1000000) -> float:
    start_time = time.time()
    for _ in range(iterations):
        shape.area()
    return time.time() - start_time

# Compare method resolution time for different inheritance depths
circle_time = measure_method_resolution(Circle(5, "red"))
rectangle_time = measure_method_resolution(Rectangle(4, 6, "green"))

print("\nMethod Resolution Time Comparison:")
print(f"Circle (depth 1): {circle_time:.6f} seconds")
print(f"Rectangle (depth 2): {rectangle_time:.6f} seconds")
print(f"Overhead ratio: {rectangle_time/circle_time:.2f}x")


In [ ]:
# 3. Encapsulation and Data Hiding

## Example: Library Management System

### Class Diagram
```
+------------------+
|     Library      |
+------------------+
| - _books        |
| - _members      |
| - _transactions |
+------------------+
| + add_book()    |
| + remove_book() |
| + search_book() |
| + issue_book()  |
| + return_book() |
+------------------+
        ↓
+------------------+
|       Book       |
+------------------+
| - _id           |
| - _title        |
| - _author       |
| - _status       |
+------------------+
| + get_info()    |
| + is_available()|
+------------------+

Performance Considerations:
- Book search: O(log n) with indexed storage
- Transaction logging: O(1) amortized
- Member validation: O(1) with hash table
```

### Key Features:
- Private attributes with property decorators
- Validation and error handling
- Event logging and tracking
- Data integrity checks


In [ ]:
from typing import Dict, List, Optional
from datetime import datetime, timedelta
from enum import Enum
import uuid

class BookStatus(Enum):
    AVAILABLE = "available"
    ISSUED = "issued"
    MAINTENANCE = "maintenance"

class Book:
    def __init__(self, title: str, author: str):
        self._id = str(uuid.uuid4())
        self._title = title
        self._author = author
        self._status = BookStatus.AVAILABLE
        self._issue_date: Optional[datetime] = None
        self._return_date: Optional[datetime] = None
    
    @property
    def id(self) -> str:
        return self._id
    
    @property
    def title(self) -> str:
        return self._title
    
    @property
    def status(self) -> BookStatus:
        return self._status
    
    @property
    def is_available(self) -> bool:
        return self._status == BookStatus.AVAILABLE
    
    def issue_to_member(self, issue_period: int = 14) -> None:
        if not self.is_available:
            raise ValueError("Book is not available for issue")
        self._status = BookStatus.ISSUED
        self._issue_date = datetime.now()
        self._return_date = self._issue_date + timedelta(days=issue_period)
    
    def return_book(self) -> None:
        if self._status != BookStatus.ISSUED:
            raise ValueError("Book is not currently issued")
        self._status = BookStatus.AVAILABLE
        self._issue_date = None
        self._return_date = None
    
    def get_info(self) -> Dict:
        return {
            "id": self._id,
            "title": self._title,
            "author": self._author,
            "status": self._status.value,
            "issue_date": self._issue_date,
            "return_date": self._return_date
        }

class Library:
    def __init__(self):
        self._books: Dict[str, Book] = {}
        self._transactions: List[Dict] = []
    
    def add_book(self, title: str, author: str) -> Book:
        book = Book(title, author)
        self._books[book.id] = book
        self._log_transaction("ADD_BOOK", book.id)
        return book
    
    def remove_book(self, book_id: str) -> None:
        if book_id not in self._books:
            raise ValueError("Book not found")
        book = self._books[book_id]
        if not book.is_available:
            raise ValueError("Cannot remove book that is currently issued")
        del self._books[book_id]
        self._log_transaction("REMOVE_BOOK", book_id)
    
    def search_book(self, title: str) -> List[Book]:
        return [book for book in self._books.values() 
                if title.lower() in book.title.lower()]
    
    def issue_book(self, book_id: str, member_id: str) -> None:
        if book_id not in self._books:
            raise ValueError("Book not found")
        book = self._books[book_id]
        book.issue_to_member()
        self._log_transaction("ISSUE_BOOK", book_id, member_id)
    
    def return_book(self, book_id: str) -> None:
        if book_id not in self._books:
            raise ValueError("Book not found")
        book = self._books[book_id]
        book.return_book()
        self._log_transaction("RETURN_BOOK", book_id)
    
    def _log_transaction(self, action: str, book_id: str, member_id: str = None) -> None:
        transaction = {
            "timestamp": datetime.now(),
            "action": action,
            "book_id": book_id,
            "member_id": member_id
        }
        self._transactions.append(transaction)
    
    def get_transaction_history(self) -> List[Dict]:
        return self._transactions.copy()

# Example usage and demonstration
library = Library()

# Add books
book1 = library.add_book("Python Programming", "John Smith")
book2 = library.add_book("Data Structures", "Jane Doe")
book3 = library.add_book("Algorithms", "John Smith")

# Search for books
print("Search Results for 'Python':")
for book in library.search_book("Python"):
    print(book.get_info())

# Issue and return books
try:
    library.issue_book(book1.id, "member123")
    print("\nBook issued successfully")
    print(book1.get_info())
    
    library.return_book(book1.id)
    print("\nBook returned successfully")
    print(book1.get_info())
except ValueError as e:
    print(f"Error: {e}")

# View transaction history
print("\nTransaction History:")
for transaction in library.get_transaction_history():
    print(f"{transaction['action']}: {transaction['timestamp']}")


In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Optional
from datetime import datetime
import uuid

class PaymentStatus(Enum):
    PENDING = "pending"
    COMPLETED = "completed"
    FAILED = "failed"
    REFUNDED = "refunded"

class PaymentMethod(ABC):
    def __init__(self):
        self._transactions: Dict[str, Dict] = {}
    
    @abstractmethod
    def process_payment(self, amount: float, currency: str) -> str:
        """Process a payment and return transaction ID."""
        pass
    
    @abstractmethod
    def refund(self, transaction_id: str) -> bool:
        """Refund a payment."""
        pass
    
    def get_transaction(self, transaction_id: str) -> Optional[Dict]:
        """Get transaction details."""
        return self._transactions.get(transaction_id)
    
    def _create_transaction(self, amount: float, currency: str, 
                          payment_type: str) -> str:
        """Create a new transaction record."""
        transaction_id = str(uuid.uuid4())
        transaction = {
            "id": transaction_id,
            "amount": amount,
            "currency": currency,
            "type": payment_type,
            "status": PaymentStatus.PENDING.value,
            "timestamp": datetime.now(),
            "refunded": False
        }
        self._transactions[transaction_id] = transaction
        return transaction_id

class CreditCardPayment(PaymentMethod):
    def __init__(self, card_number: str, expiry: str, cvv: str):
        super().__init__()
        self._validate_card(card_number, expiry, cvv)
        self._card_number = card_number[-4:]  # Store only last 4 digits
        self._expiry = expiry
    
    def _validate_card(self, card: str, expiry: str, cvv: str) -> None:
        if not card.isdigit() or len(card) not in [15, 16]:
            raise ValueError("Invalid card number")
        if not cvv.isdigit() or len(cvv) not in [3, 4]:
            raise ValueError("Invalid CVV")
        # Add more validation as needed
    
    def process_payment(self, amount: float, currency: str) -> str:
        transaction_id = self._create_transaction(amount, currency, "credit_card")
        try:
            # Simulate payment processing
            self._transactions[transaction_id]["status"] = PaymentStatus.COMPLETED.value
            return transaction_id
        except Exception as e:
            self._transactions[transaction_id]["status"] = PaymentStatus.FAILED.value
            raise ValueError(f"Payment failed: {str(e)}")
    
    def refund(self, transaction_id: str) -> bool:
        transaction = self.get_transaction(transaction_id)
        if not transaction:
            raise ValueError("Transaction not found")
        if transaction["status"] != PaymentStatus.COMPLETED.value:
            raise ValueError("Cannot refund non-completed transaction")
        if transaction["refunded"]:
            raise ValueError("Transaction already refunded")
        
        transaction["refunded"] = True
        transaction["status"] = PaymentStatus.REFUNDED.value
        return True

class DigitalPayment(PaymentMethod):
    def __init__(self, provider: str):
        super().__init__()
        self.provider = provider
    
    @abstractmethod
    def connect_to_provider(self) -> bool:
        """Connect to payment provider."""
        pass

class PayPalPayment(DigitalPayment):
    def __init__(self, email: str):
        super().__init__("PayPal")
        self._email = email
    
    def connect_to_provider(self) -> bool:
        # Simulate connection to PayPal
        return True
    
    def process_payment(self, amount: float, currency: str) -> str:
        if not self.connect_to_provider():
            raise ConnectionError("Failed to connect to PayPal")
        
        transaction_id = self._create_transaction(amount, currency, "paypal")
        try:
            # Simulate PayPal payment processing
            self._transactions[transaction_id]["status"] = PaymentStatus.COMPLETED.value
            return transaction_id
        except Exception as e:
            self._transactions[transaction_id]["status"] = PaymentStatus.FAILED.value
            raise ValueError(f"PayPal payment failed: {str(e)}")
    
    def refund(self, transaction_id: str) -> bool:
        if not self.connect_to_provider():
            raise ConnectionError("Failed to connect to PayPal")
        
        transaction = self.get_transaction(transaction_id)
        if not transaction:
            raise ValueError("Transaction not found")
        
        transaction["refunded"] = True
        transaction["status"] = PaymentStatus.REFUNDED.value
        return True

# Example usage and demonstration
def process_order(payment_method: PaymentMethod, amount: float, currency: str = "USD"):
    try:
        transaction_id = payment_method.process_payment(amount, currency)
        print(f"Payment processed successfully. Transaction ID: {transaction_id}")
        
        # Get transaction details
        transaction = payment_method.get_transaction(transaction_id)
        print(f"Transaction details: {transaction}")
        
        # Demonstrate refund
        payment_method.refund(transaction_id)
        print(f"Refund processed successfully")
        
        # Get updated transaction details
        updated_transaction = payment_method.get_transaction(transaction_id)
        print(f"Updated transaction details: {updated_transaction}")
        
    except Exception as e:
        print(f"Error: {str(e)}")

# Test with different payment methods
credit_card = CreditCardPayment("4111111111111111", "12/25", "123")
paypal = PayPalPayment("user@example.com")

print("Processing credit card payment:")
process_order(credit_card, 99.99)

print("\nProcessing PayPal payment:")
process_order(paypal, 49.99)


In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Set
from datetime import datetime
import threading
import time
import random

class Observer(ABC):
    @abstractmethod
    def update(self, subject: 'Subject') -> None:
        """Receive update from subject."""
        pass
    
    @abstractmethod
    def get_state(self) -> Dict:
        """Get current state of the observer."""
        pass

class Subject(ABC):
    def __init__(self):
        self._observers: Set[Observer] = set()
        self._lock = threading.Lock()
    
    def attach(self, observer: Observer) -> None:
        """Attach an observer to the subject."""
        with self._lock:
            self._observers.add(observer)
    
    def detach(self, observer: Observer) -> None:
        """Detach an observer from the subject."""
        with self._lock:
            self._observers.discard(observer)
    
    def notify(self) -> None:
        """Notify all observers about an update."""
        with self._lock:
            for observer in self._observers:
                observer.update(self)

class StockMarket(Subject):
    def __init__(self):
        super().__init__()
        self._stocks: Dict[str, float] = {}
        self._last_update = datetime.now()
    
    def add_stock(self, symbol: str, price: float) -> None:
        """Add a new stock to the market."""
        with self._lock:
            self._stocks[symbol] = price
            self._last_update = datetime.now()
            self.notify()
    
    def update_price(self, symbol: str, price: float) -> None:
        """Update the price of a stock."""
        if symbol not in self._stocks:
            raise ValueError(f"Stock {symbol} not found")
        with self._lock:
            self._stocks[symbol] = price
            self._last_update = datetime.now()
            self.notify()
    
    def get_price(self, symbol: str) -> float:
        """Get the current price of a stock."""
        return self._stocks.get(symbol, 0.0)
    
    def get_state(self) -> Dict:
        """Get current state of the stock market."""
        return {
            "stocks": self._stocks.copy(),
            "last_update": self._last_update
        }

class Investor(Observer):
    def __init__(self, name: str, watched_stocks: List[str]):
        self.name = name
        self.watched_stocks = set(watched_stocks)
        self.portfolio: Dict[str, Dict] = {}
    
    def update(self, subject: Subject) -> None:
        """Receive update from the stock market."""
        if isinstance(subject, StockMarket):
            market_state = subject.get_state()
            for symbol in self.watched_stocks:
                if symbol in market_state["stocks"]:
                    current_price = market_state["stocks"][symbol]
                    if symbol in self.portfolio:
                        self.portfolio[symbol]["current_price"] = current_price
                        self.portfolio[symbol]["profit"] = (
                            current_price - self.portfolio[symbol]["buy_price"]
                        )
    
    def buy_stock(self, symbol: str, shares: int, price: float) -> None:
        """Buy shares of a stock."""
        self.portfolio[symbol] = {
            "shares": shares,
            "buy_price": price,
            "current_price": price,
            "profit": 0.0
        }
        self.watched_stocks.add(symbol)
    
    def get_state(self) -> Dict:
        """Get current state of the investor's portfolio."""
        return {
            "name": self.name,
            "portfolio": self.portfolio.copy(),
            "total_profit": sum(stock["profit"] * stock["shares"] 
                              for stock in self.portfolio.values())
        }

class Analytics(Observer):
    def __init__(self):
        self.price_history: Dict[str, List[Dict]] = {}
        self.statistics: Dict[str, Dict] = {}
    
    def update(self, subject: Subject) -> None:
        """Record and analyze price updates."""
        if isinstance(subject, StockMarket):
            market_state = subject.get_state()
            timestamp = market_state["last_update"]
            
            for symbol, price in market_state["stocks"].items():
                if symbol not in self.price_history:
                    self.price_history[symbol] = []
                
                self.price_history[symbol].append({
                    "timestamp": timestamp,
                    "price": price
                })
                
                # Update statistics
                prices = [record["price"] for record in self.price_history[symbol]]
                self.statistics[symbol] = {
                    "min": min(prices),
                    "max": max(prices),
                    "avg": sum(prices) / len(prices)
                }
    
    def get_state(self) -> Dict:
        """Get current analytics state."""
        return {
            "statistics": self.statistics.copy(),
            "history_length": {symbol: len(history) 
                             for symbol, history in self.price_history.items()}
        }

# Example usage and demonstration
def simulate_market():
    # Create stock market and observers
    market = StockMarket()
    investor = Investor("John Doe", ["AAPL", "GOOGL"])
    analytics = Analytics()
    
    # Attach observers
    market.attach(investor)
    market.attach(analytics)
    
    # Initial stock setup
    market.add_stock("AAPL", 150.0)
    market.add_stock("GOOGL", 2800.0)
    market.add_stock("MSFT", 300.0)
    
    # Investor buys some stocks
    investor.buy_stock("AAPL", 10, 150.0)
    investor.buy_stock("GOOGL", 5, 2800.0)
    
    # Simulate price changes
    for _ in range(5):
        for symbol in ["AAPL", "GOOGL", "MSFT"]:
            current_price = market.get_price(symbol)
            change = random.uniform(-10, 10)
            market.update_price(symbol, current_price + change)
        time.sleep(1)  # Simulate time passing
    
    # Print final states
    print("\nFinal Investor State:")
    print(investor.get_state())
    
    print("\nFinal Analytics State:")
    print(analytics.get_state())

# Run simulation
simulate_market()
